# Load data and variables


In [ ]:
import pandas as pd
import os
#load data from file

direct = os.getcwd()

path = direct + "\\data\\vos-instances\\totals.csv"
df = pd.read_csv(path)
# transform columns to int
df.head(50)

In [41]:
# Get countries shapes
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import MultiPolygon, Polygon


# Load medium-resolution shapefiles for the world and filter for Netherlands, Belgium, and Luxembourg
world  = gpd.read_file(direct + '\\data\\ne_50m_admin_0_countries.shp') 
countries = world[ (world['NAME'] == 'Belgium') | (world['NAME'] == 'Luxembourg')]

# Assuming 'world' is already loaded and contains the necessary shapefile data
netherlands = world[world['NAME'] == 'Netherlands'].copy()

# Define coordinate boundaries (for example, excluding territories outside mainland Europe)
min_latitude = 50.75  # Northern limit of mainland Europe approx
max_latitude = 54
min_longitude = 3.35  # Western limit of mainland Europe approx
max_longitude = 7.22

# Filter polygons based on coordinate boundaries
def within_bounds(polygon, min_lat, max_lat, min_lon, max_lon):
    """ Check if the polygon falls within the specified lat/lon bounds """
    x, y = polygon.exterior.coords.xy
    return all(min_lon <= lon <= max_lon for lon in x) and all(min_lat <= lat <= max_lat for lat in y)

# Iterate over each polygon in the MultiPolygon
if isinstance(netherlands.geometry.iloc[0], MultiPolygon):
    filtered_polygons = [polygon for polygon in netherlands.geometry.iloc[0].geoms 
                         if within_bounds(polygon, min_latitude, max_latitude, min_longitude, max_longitude)]
else:
    # If it's not a MultiPolygon (unlikely in this case), handle it as a single Polygon
    filtered_polygons = [netherlands.geometry.iloc[0]] if within_bounds(netherlands.geometry.iloc[0], min_latitude, max_latitude, min_longitude, max_longitude) else []

# Create a new geometry from the filtered polygons
new_geometry = MultiPolygon(filtered_polygons) if len(filtered_polygons) > 1 else (filtered_polygons[0] if filtered_polygons else None)

# Update the geometry in the GeoDataFrame
netherlands.at[netherlands.index[0], 'geometry'] = new_geometry

# Use concat to merge GeoDataFrames correctly
countries = gpd.GeoDataFrame(pd.concat([countries, netherlands], ignore_index=True))

In [ ]:
from custom.PC_Class import PC
depots = {"Woerden": {"depotId": 0,
                      "PC": "NL3447GP"}, 
          "Eindhoven": {"depotId": 1,
                        "PC": "NL5644RL"}, 
          "Beilen": {"depotId": 2,
                     "PC": "NL9411TX"}, 
          "Bornem": {"depotId": 3,
                     "PC": "BE2880"}
          }
PC_obj = PC()

for depot in depots:
    lat, lon = PC_obj.get_coordinates(depots[depot]["PC"])
    depots[depot]["lat"] = lat
    depots[depot]["lon"] = lon

# Plot whole year


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import geopandas as gpd
import seaborn as sns
import pandas as pd
import numpy as np


color_mapping = {
    (0,): 'red',       # Red
    (1,): 'blue',      # Blue
    (2,): 'cyan',      # Cyan
    (3,): 'gold',    # Yellow
    (0, 1): 'purple',  # Red + Blue
    (1, 3): 'green',   # Blue + Yellow
    (0, 3): 'darkorange',  # Red + Yellow
    (0, 2): 'magenta', # Red + Cyan
    (1, 2): 'teal',   # Blue + Cyan
    (2, 3): 'lime',    # Cyan + Yellow
    'black': 'black'   # More than two colors overlap
}


# Group orders by delivery locations and create a new column with all unique values for idDepot
dfg = df.groupby(['latitude', 'longitude'])['idDepot'].apply(lambda x: tuple(sorted(set(x)))).reset_index()

# Define a function to determine the color based on the tuple of idDepot values
def determine_color(ids):
    if len(ids) == 1:
        return color_mapping[ids]
    elif len(ids) == 2:
        return color_mapping.get(ids, 'black')
    else:
        return 'black'

# Apply the function to determine the color
dfg['color'] = dfg['idDepot'].apply(determine_color)

# Add a new column 'plot_order' to prioritize mixed colors in the plotting
dfg['plot_order'] = dfg['idDepot'].apply(lambda x: 0 if len(x) == 1 else 1)
dfg = dfg.sort_values(by='plot_order',ascending=True)

# Plot the base map with the outlines of the countries
fig, ax = plt.subplots(figsize=(10, 10))
# fig.patch.set_facecolor('lightgrey')  # Set the outer background to grey
# ax.set_facecolor('lightgrey')         # Set the inner plot area to grey
countries.boundary.plot(ax=ax, linewidth=1, edgecolor='black')

# Plot the scatter plot with custom colors
scatter = ax.scatter(dfg['longitude'], dfg['latitude'], c=dfg['color'] , alpha=1, s=5)

# Existing depot plotting code here...
for depot_name, depot_info in depots.items():
    ax.scatter(depot_info['lon'], depot_info['lat'], 
               s=100,  # Size of the hexagon marker
               c=color_mapping[(depot_info['depotId'],)],  # Color from the color_mapping dictionary
               marker='h',  # Hexagon shape
               edgecolors='black',  # Black edge color
               label=depot_name)  # Label for legend

# Create a custom legend entry for the black dot
black_dot = mlines.Line2D([], [], color='black', marker='o', linestyle='None',
                          markersize=5, label='More than two delivery depots')

# Collect all legend handles and labels from the plot
handles, labels = ax.get_legend_handles_labels()

# Add the custom black dot legend entry
handles.append(black_dot)
labels.append('More than 2 delivery depots')

# Create the legend with the updated handles and labels
ax.legend(handles=handles, labels=labels, title="Legend", loc='lower left', fontsize=13)

# Customize plot
ax.set_ylabel('Latitude', fontsize=14)
ax.set_xlabel('Longitude', fontsize=14)
plt.title('Delivery Locations and their Delivery Depots', fontsize=16)
# plt.xlim(2.5, 7.3) 
# plt.ylim(49.4, 53.7) 

ax.set_xticks(np.arange(3, 8, 1))
ax.set_yticks(np.arange(50, 54, 1))

plt.show()

In [ ]:
dfg = df.groupby(['latitude', 'longitude'])['idDepot'].apply(lambda x: (list(x))).reset_index()
dfg['majority_depot'] = dfg['idDepot'].apply(lambda x: max(set(x), key=x.count))

color_mapping = {
    0: 'red',       # Red
    1: 'blue',      # Blue
    2: 'cyan',      # Cyan
    3: 'gold',    # Yellow
    4: 'black'}
dfg['color'] = dfg['majority_depot'].apply(lambda x: color_mapping[x])



# Plot the base map with the outlines of the countries
fig, ax = plt.subplots(figsize=(10, 10))
# fig.patch.set_facecolor('lightgrey')  # Set the outer background to grey
# ax.set_facecolor('lightgrey')         # Set the inner plot area to grey
countries.boundary.plot(ax=ax, linewidth=1, edgecolor='black')

# Plot the scatter plot with custom colors
scatter = ax.scatter(dfg['longitude'], dfg['latitude'], c=dfg['color'] , alpha=0.5, s=5)

# Existing depot plotting code here...
for depot_name, depot_info in depots.items():
    ax.scatter(depot_info['lon'], depot_info['lat'], 
               s=100,  # Size of the hexagon marker
               c=color_mapping[(depot_info['depotId'])],  # Color from the color_mapping dictionary
               marker='h',  # Hexagon shape
               edgecolors='black',  # Black edge color
               label=depot_name)  # Label for legend

# Create a custom legend entry for the black dot
# black_dot = mlines.Line2D([], [], color='black', marker='o', linestyle='None',
#                           markersize=5, label='More than two delivery depots')

# Collect all legend handles and labels from the plot
handles, labels = ax.get_legend_handles_labels()

# Add the custom black dot legend entry
# handles.append(black_dot)
# labels.append('More than 2 delivery depots')

# Create the legend with the updated handles and labels
ax.legend(handles=handles, labels=labels, title="Legend", loc='lower left', fontsize=14)

ax.set_ylabel('Latitude', fontsize=14)
ax.set_xlabel('Longitude', fontsize=14)
plt.title('Delivery Depots based on Majority Vote', fontsize=16)
plt.xlim(2.5, 7.3) 
plt.ylim(49.4, 53.7) 

ax.set_xticks(np.arange(3, 8, 1))
ax.set_yticks(np.arange(50, 54, 1))

plt.show()

# Create boundaries


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import pandas as pd

# Scaling features

# Group by latitude and longitude and calculate both the unique depot IDs and the sum of qDel
# Group by latitude and longitude and calculate both the unique depot IDs and the sum of qDel
dfg = df.groupby(['latitude', 'longitude', 'idDepot']).agg({
    'qDel': 'sum'  # Summing up the qDel for the grouped coordinates
}).reset_index()

color_mapping = {
    0: 'red',       # Red
    1: 'blue',      # Blue
    2: 'cyan',      # Cyan
    3: 'gold',    # Yellow
    4: 'black'}


dfg['color'] = dfg['idDepot'].apply(lambda x: color_mapping[x])

# Create an SVM model for multiclass classification
# clf = svm.SVC(kernel='rbf', C=1, decision_function_shape='ovo')
clf = svm.SVC(kernel='linear', decision_function_shape='ovo')
clf.fit(dfg[['longitude', 'latitude']], dfg['idDepot'], sample_weight=dfg['qDel'])

print("learned the SVM")



In [ ]:
# Plotting decision boundaries
x_min, x_max = 2.5, 7.3
y_min, y_max = 49.4, 53.7

def make_meshgrid(x_min, x_max, y_min, y_max, h=.005):
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy

def calculcate_contour(clf, xx, yy):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    return Z

def plot_contours(ax, xx, yy, Z ,**params):
    out = ax.contourf(xx, yy, Z, **params)
    return out

xx, yy = make_meshgrid(x_min, x_max, y_min, y_max)


Z = calculcate_contour(clf, xx, yy)

In [ ]:
from matplotlib.colors import ListedColormap

color_mapping = {
    0: 'red',       # Red
    1: 'blue',      # Blue
    2: 'white',      # Cyan
    3: 'gold',    # Yellow
    4: 'black'}

colors = [color_mapping[i] for i in sorted(color_mapping.keys())]
custom_cmap = ListedColormap(colors)

fig, ax = plt.subplots(figsize=(10, 10))
countries.boundary.plot(ax=ax, linewidth=1, edgecolor='black')
plot_contours(ax, xx, yy, Z ,cmap=custom_cmap, alpha=0.5)


# Mask areas outside the country boundaries
# def mask_outside_polygons(shape, ax):
#     if isinstance(shape, MultiPolygon):
#         for polygon in shape.geoms:
#             xs, ys = polygon.exterior.xy
#             ax.fill(xs, ys, 'white', edgecolor='none')
#     elif isinstance(shape, Polygon):
#         xs, ys = shape.exterior.xy
#         ax.fill(xs, ys, 'white', edgecolor='none')

# # Apply masking to each country
# for index, row in countries.iterrows():
#     mask_outside_polygons(row['geometry'], ax)

dfg_majority = df.groupby(['latitude', 'longitude'])['idDepot'].apply(lambda x: (list(x))).reset_index()
dfg_majority['majority_depot'] = dfg_majority['idDepot'].apply(lambda x: max(set(x), key=x.count))
color_mapping = {
    0: 'red',       # Red
    1: 'blue',      # Blue
    2: 'cyan',      # Cyan
    3: 'gold',    # Yellow
    4: 'black'}
dfg_majority['color'] = dfg_majority['majority_depot'].apply(lambda x: color_mapping[x])



# Plot the scatter plot with custom colors
scatter = ax.scatter(dfg_majority['longitude'], dfg_majority['latitude'], c=dfg_majority['color'] , alpha=1, s=2)

# Existing depot plotting code here...
for depot_name, depot_info in depots.items():
    ax.scatter(depot_info['lon'], depot_info['lat'], 
               s=100,  # Size of the hexagon marker
               c=color_mapping[(depot_info['depotId'])],  # Color from the color_mapping dictionary
               marker='h',  # Hexagon shape
               edgecolors='black',  # Black edge color
               label=depot_name)  # Label for legend



plt.xlim(x_min, x_max) 
plt.ylim(y_min, y_max) 

ax.set_xticks(np.arange(3, 8, 1))
ax.set_yticks(np.arange(50, 54, 1))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, title="Legend", loc='upper left', fontsize=14)

ax.set_ylabel('Latitude', fontsize=14)
ax.set_xlabel('Longitude', fontsize=14)
plt.title('Fixed Delivery Depots for Locations', fontsize=16)
plt.show()

In [ ]:
color_mapping = {
    0: 'red',       # Red
    1: 'blue',      # Blue
    2: 'cyan',      # Cyan
    3: 'gold',    # Yellow
    4: 'black'}

colors = [color_mapping[i] for i in sorted(color_mapping.keys())]
custom_cmap = ListedColormap(colors)


fig, ax = plt.subplots(figsize=(10, 10))
countries.boundary.plot(ax=ax, linewidth=1, edgecolor='black')
plot_contours(ax, xx, yy, Z ,cmap=custom_cmap, alpha=0.5)

# Existing depot plotting code here...
for depot_name, depot_info in depots.items():
    ax.scatter(depot_info['lon'], depot_info['lat'], 
               s=100,  # Size of the hexagon marker
               c=color_mapping[(depot_info['depotId'])],  # Color from the color_mapping dictionary
               marker='h',  # Hexagon shape
               edgecolors='black',  # Black edge color
               label=depot_name)  # Label for legend



plt.xlim(x_min, x_max) 
plt.ylim(y_min, y_max) 

ax.set_xticks(np.arange(3, 8, 1))
ax.set_yticks(np.arange(50, 54, 1))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, title="Legend", loc='upper left', fontsize=14)


ax.set_ylabel('Latitude', fontsize=14)
ax.set_xlabel('Longitude', fontsize=14)
plt.title('Fixed Delivery Regions for Depot Locations', fontsize=16)
plt.show()

# Compare fixed locations with eventual locations

In [71]:
df["Fixed_idDepot"] = clf.predict(df[['longitude', 'latitude']])

In [ ]:
# percentage of orders which are assigned to the correct depot
correct_depot = df[df["idDepot"] == df["Fixed_idDepot"]].shape[0] / df.shape[0]
print(f"Percentage of orders assigned to the correct depot when using fixed regions: {correct_depot * 100:.2f}%")

# percentage of volume which are assigned to the correct depot
correct_depot_volume = df[df["idDepot"] == df["Fixed_idDepot"]]["qDel"].sum() / df["qDel"].sum()
print(f"Percentage of volume assigned to the correct depot when using fixed regions: {correct_depot_volume * 100:.2f}%")


In [ ]:
from matplotlib.colors import ListedColormap

color_mapping = {
    0: 'red',       # Red
    1: 'blue',      # Blue
    2: 'white',      # Cyan
    3: 'gold',    # Yellow
    4: 'black'}

colors = [color_mapping[i] for i in sorted(color_mapping.keys())]
custom_cmap = ListedColormap(colors)

fig, ax = plt.subplots(figsize=(10, 10))
countries.boundary.plot(ax=ax, linewidth=1, edgecolor='black')
plot_contours(ax, xx, yy, Z ,cmap=custom_cmap, alpha=0.5)


# Mask areas outside the country boundaries
# def mask_outside_polygons(shape, ax):
#     if isinstance(shape, MultiPolygon):
#         for polygon in shape.geoms:
#             xs, ys = polygon.exterior.xy
#             ax.fill(xs, ys, 'white', edgecolor='none')
#     elif isinstance(shape, Polygon):
#         xs, ys = shape.exterior.xy
#         ax.fill(xs, ys, 'white', edgecolor='none')

# # Apply masking to each country
# for index, row in countries.iterrows():
#     mask_outside_polygons(row['geometry'], ax)

dfg_error = df[df["idDepot"] != df["Fixed_idDepot"]].groupby(['latitude', 'longitude'])['idDepot'].apply(lambda x: (list(x))).reset_index()
dfg_error['majority_depot'] = dfg_error['idDepot'].apply(lambda x: max(set(x), key=x.count))
color_mapping = {
    0: 'red',       # Red
    1: 'blue',      # Blue
    2: 'cyan',      # Cyan
    3: 'gold',    # Yellow
    4: 'black'}
dfg_error['color'] = dfg_error['majority_depot'].apply(lambda x: color_mapping[x])
# dfg_error = dfg_error[dfg_error["majority_depot"] == 2]


# Plot the scatter plot with custom colors
scatter = ax.scatter(dfg_error['longitude'], dfg_error['latitude'], c=dfg_error['color'] , alpha=1, s=2)

# Existing depot plotting code here...
for depot_name, depot_info in depots.items():
    ax.scatter(depot_info['lon'], depot_info['lat'], 
               s=100,  # Size of the hexagon marker
               c=color_mapping[(depot_info['depotId'])],  # Color from the color_mapping dictionary
               marker='h',  # Hexagon shape
               edgecolors='black',  # Black edge color
               label=depot_name)  # Label for legend



plt.xlim(x_min, x_max) 
plt.ylim(y_min, y_max) 

ax.set_xticks(np.arange(3, 8, 1))
ax.set_yticks(np.arange(50, 54, 1))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, title="Legend", loc='upper left', fontsize=14)

ax.set_ylabel('Latitude', fontsize=14)
ax.set_xlabel('Longitude', fontsize=14)
plt.title('Fixed Delivery Depots for Locations', fontsize=16)
plt.show()




## Order size comparison \ Wrong depot allocation


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
countries.boundary.plot(ax=ax, linewidth=1, edgecolor='black')
plot_contours(ax, xx, yy, Z ,cmap=custom_cmap, alpha=0.5)

# Plot the scatter plot with custom colors
scatter = ax.scatter(df_wrong['longitude'], df_wrong['latitude'], alpha=1, s=2)

# Existing depot plotting code here...
for depot_name, depot_info in depots.items():
    ax.scatter(depot_info['lon'], depot_info['lat'], 
               s=100,  # Size of the hexagon marker
               c=color_mapping[(depot_info['depotId'])],  # Color from the color_mapping dictionary
               marker='h',  # Hexagon shape
               edgecolors='black',  # Black edge color
               label=depot_name)  # Label for legend



plt.xlim(x_min, x_max) 
plt.ylim(y_min, y_max) 

ax.set_xticks(np.arange(3, 8, 1))
ax.set_yticks(np.arange(50, 54, 1))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, title="Legend", loc='upper left')

ax.set_ylabel('Longitude')
ax.set_xlabel('Latitude')
plt.title('Fixed Delivery Depots for Locations')
plt.show()

In [ ]:
df_wrong["dist_from_hub"] = df_wrong.apply(lambda x: PC_obj.haversine_distance([depot for depot in depots.values() if depot["depotId"] == x['idDepot']][0]["lat"], [depot for depot in depots.values() if depot["depotId"] == x['idDepot']][0]["lon"], x["latitude"], x["longitude"]), axis=1)

In [ ]:
# Sorting the DataFrame based on the distance from the hub
df_sorted = df_wrong.sort_values(by='dist_from_hub')

# Applying a rolling average for smoothing: window size can be adjusted
window_size = 1000  # Adjust this based on how smooth you want the line to be
df_sorted['qDel_smoothed'] = df_sorted['qDel'].rolling(window=window_size, center=True).mean()

# Plotting
plt.figure(figsize=(10, 6))  # You can adjust the figure size as needed
# plt.plot(df_sorted['dist_from_hub'], df_sorted['qDel'], marker='o', alpha=0.5, label='Original')  # Original data
plt.plot(df_sorted['dist_from_hub'], df_sorted['qDel_smoothed'], marker='o', label='Smoothed')  # Smoothed data
plt.title('Relationship between Distance from Hub and qDel with Smoothing')
plt.xlabel('Distance from Hub')
plt.ylabel('qDel')
plt.grid(True)  # Adds a grid for easier visualization
plt.legend()  # Adds a legend to distinguish between original and smoothed data
plt.show()

In [ ]:
# Creating bins of 10 km
bins = pd.interval_range(start=0, freq=10, end=df_sorted['dist_from_hub'].max() + 10)
df_sorted['Distance_bin'] = pd.cut(df_sorted['dist_from_hub'], bins=bins)

# Plotting the boxplot for each bin
plt.figure(figsize=(12, 8))
df_sorted.boxplot(column='qDel', by='Distance_bin', grid=True)

plt.title('Boxplot of qDel for Every 10km Distance from Hub')
plt.suptitle('')  # Suppress the automatic title to clean up the plot
plt.xlabel('Distance from Hub (km)')
plt.ylabel('qDel')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.show()

# Create compare scenarios

### Load all years data

In [ ]:
from custom.GeoSpatialEncoder import GeoSpatialEncoder
from custom.PC_Class import PC
import importlib
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
import os
datetime_cols = ['CREATIONDATETIME', 'LAAD_DATETIME_VAN', 'LAAD_DATETIME_TOT', 'LOS_DATETIME_VAN', 'LOS_DATETIME_TOT', '15CREATIONDATETIME']
direct = os.getcwd()
file_path = direct + "////data////vos_input_data////MultiHubData3_cleaned.csv" 
total_rows = sum(1 for row in open(file_path, 'r', encoding='utf-8'))
chunk_size = 10000  
tqdm.pandas(desc="Reading CSV")
chunks = pd.read_csv(file_path, chunksize=chunk_size, iterator=True, index_col = 0, parse_dates=datetime_cols)

df_orders = pd.concat(tqdm(chunks, total=total_rows//chunk_size))

# Convert the 'LOS_DATETIME_VAN' column to datetime format
for column in datetime_cols:
    print(f"column: {column}")
    df_orders[column] = pd.to_datetime(df_orders[column], errors='coerce')

print("Lenght of input data:", str(len(df_orders)))
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

def write_to_csv(df, file_name):
    path = # PATH TO SOLUTIONS HERE
    
    file_name = path + file_name
    df.to_csv(file_name)

#### Create Scenarios

In [17]:
from custom.DataCreator import Scenario
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names, but SVC was fitted with feature names")

path = # PATH TO SOLUTIONS HERE
files = os.listdir(path)
files = [files.replace(".csv", "") for files in files if ".csv" in files]
for instance_name in files:
    scenario = Scenario(path, instance_name, PC_obj)
    scenario.instance_name = instance_name+"_free"
    scenario.write_instance()
    for order in scenario.orders:
        scenario.orders[order]["idDepot"] = int(clf.predict([[scenario.orders[order]["longitude"], scenario.orders[order]["latitude"]]])[0])
    scenario.instance_name = instance_name+"_fixed"
    scenario.write_instance(include_linehauls=False)

#### Calculate for each fixed scenario the additional cost for fixed


In [ ]:
units_per_pallet = 100
linehaul_capacity = # CAPACITY OF LINEHAUL HERE
linehaul_km = # COST PER KM OF LINEHAUL HERE
crossdock_cost = # COST OF CROSSDOCK HERE
linehauls = {}

for depot1 in depots:
    for depot2 in depots:
        d1 = depots[depot1]['depotId']
        d2 = depots[depot2]['depotId']
        if d1 < d2:
            linehauls[(d1, d2)] = {"distance": PC_obj.get_distance(depots[depot1]["PC"], depots[depot2]["PC"])}
            linehauls[(d1, d2)]["cost"] = (linehauls[(d1, d2)]["distance"] * (linehaul_km /linehaul_capacity))+ crossdock_cost
        elif d1 > d2:
            linehauls[(d1, d2)] = {"distance": PC_obj.get_distance(depots[depot1]["PC"], depots[depot2]["PC"])}
            linehauls[(d1, d2)]["cost"] = (linehauls[(d2, d1)]["distance"] * (linehaul_km /linehaul_capacity))+ crossdock_cost
        
        else:
            linehauls[(d1, d2)] = {"distance": 0,
                                   "cost": 0}
linehauls

In [ ]:
path = # PATH TO SOLUTIONS HERE
files = os.listdir(path)
files = [files.replace(".csv", "") for files in files if files.endswith(".csv")]

# for file in files:
for file in ['2023-09-14']:
    df_scenario = pd.read_csv(path + file + ".csv")
    df_scenario["LOSCOORDINATEN"] = df_scenario.apply(lambda x: PC_obj.get_coordinates(x['LOS_CPC']), axis=1)
    df_scenario[['latitude', 'longitude']] = pd.DataFrame(df_scenario['LOSCOORDINATEN'].tolist(), index=df_scenario.index)
    df_scenario["Fixed_idDepot"] = clf.predict(df_scenario[['longitude', 'latitude']])
    df_scenario['origin_idDepot_CPC'] = df_scenario.apply(lambda x: PC_obj.get_closest_hub(x["LAAD_CPC"], [depots[depot]['PC'] for depot in depots]), axis=1)
    df_scenario['origin_idDepot'] = df_scenario.apply(lambda x: [depots[depot]['depotId'] for depot in depots if depots[depot]['PC'] == x['origin_idDepot_CPC']][0], axis=1)

    # get the free iddepot from the corresponding entry in df

    df_scenario = df_scenario.merge(df[['SHIPMENTNUMBER', 'idDepot']], on='SHIPMENTNUMBER', how='left')
    df_scenario.rename(columns={"idDepot": "Free_idDepot"}, inplace=True)
    df_scenario['LinehaulCost'] = df_scenario.apply(lambda x: linehauls[(x['origin_idDepot'], x['Fixed_idDepot'])]["cost"] * x['PALLETPLAATSEN'] , axis=1)
    # df_scenario['Linehauled'] = df_scenario.apply(lambda x: 1 if x['idDepot'] != x['Fixed_idDepot'] else 0, axis=1)
    total_cost = df_scenario['LinehaulCost'].sum()
    print(f"Total cost for {file} is {total_cost}")


In [ ]:
df_scenario[df_scenario["Fixed_idDepot"] != df_scenario["Free_idDepot"]][['LinehaulCost', 'PALLETPLAATSEN', 'origin_idDepot', 'Fixed_idDepot', 'Free_idDepot']]

In [ ]:
from matplotlib.colors import ListedColormap

color_mapping = {
    0: 'red',       # Red
    1: 'blue',      # Blue
    2: 'white',      # Cyan
    3: 'gold',    # Yellow
    4: 'black'}

colors = [color_mapping[i] for i in sorted(color_mapping.keys())]
custom_cmap = ListedColormap(colors)

fig, ax = plt.subplots(figsize=(10, 10))
countries.boundary.plot(ax=ax, linewidth=1, edgecolor='black')
plot_contours(ax, xx, yy, Z ,cmap=custom_cmap, alpha=0.5)


# Mask areas outside the country boundaries
# def mask_outside_polygons(shape, ax):
#     if isinstance(shape, MultiPolygon):
#         for polygon in shape.geoms:
#             xs, ys = polygon.exterior.xy
#             ax.fill(xs, ys, 'white', edgecolor='none')
#     elif isinstance(shape, Polygon):
#         xs, ys = shape.exterior.xy
#         ax.fill(xs, ys, 'white', edgecolor='none')

# # Apply masking to each country
# for index, row in countries.iterrows():
#     mask_outside_polygons(row['geometry'], ax)
df_scenario['idDepot'] = df_scenario['Free_idDepot']
df_scenario = df_scenario[~df_scenario['idDepot'].isna()]
dfg_error = df_scenario[df_scenario["Fixed_idDepot"] != df_scenario["Free_idDepot"]].groupby(['latitude', 'longitude'])['idDepot'].apply(lambda x: (list(x))).reset_index()
dfg_error['majority_depot'] = dfg_error['idDepot'].apply(lambda x: max(set(x), key=x.count))
color_mapping = {
    0: 'red',       # Red
    1: 'blue',      # Blue
    2: 'cyan',      # Cyan
    3: 'gold',    # Yellow
    4: 'black'}
dfg_error['color'] = dfg_error['majority_depot'].apply(lambda x: color_mapping[x])
# dfg_error = dfg_error[dfg_error["majority_depot"] == 2]


# Plot the scatter plot with custom colors
scatter = ax.scatter(dfg_error['longitude'], dfg_error['latitude'], c=dfg_error['color'] , alpha=1, s=2)

# Existing depot plotting code here...
for depot_name, depot_info in depots.items():
    ax.scatter(depot_info['lon'], depot_info['lat'], 
               s=100,  # Size of the hexagon marker
               c=color_mapping[(depot_info['depotId'])],  # Color from the color_mapping dictionary
               marker='h',  # Hexagon shape
               edgecolors='black',  # Black edge color
               label=depot_name)  # Label for legend



plt.xlim(x_min, x_max) 
plt.ylim(y_min, y_max) 

ax.set_xticks(np.arange(3, 8, 1))
ax.set_yticks(np.arange(50, 54, 1))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, title="Legend", loc='upper left')

ax.set_ylabel('Longitude')
ax.set_xlabel('Latitude')
plt.title('Fixed Delivery Depots for Locations')
plt.show()


